In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#src = str(Path.cwd().resolve().parents[0]/ 'src')
#sys.path.append(src)
#from config.paths import ROOT, DATA, EXPERIMENTS, METADATA

In [3]:
ROOT = '/content/drive/MyDrive/dataHN'
DOC_IMGS = '/content/drive/MyDrive/dataHN/data/raw_selected/doc_imgs'
EXPERIMENTS = '/content/drive/MyDrive/dataHN/experiments'
DATA = '/content/drive/MyDrive/dataHN/data'

In [1]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


# Préparation des données

In [4]:
from ultralytics import YOLO
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image, ImageOps
from tqdm import tqdm

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [16]:
def resize_save_img(input_path, output_path, target_size=640):
    """Redimensionne une image en gardant le ratio puis la sauvegarde."""
    img = Image.open(input_path).convert('RGB')
    w, h = img.size
    if w > h:
        new_w = target_size
        new_h = int(h * target_size / w)
    else:
        new_h = target_size
        new_w = int(w * target_size / h)
    img_resized = img.resize((new_w, new_h), Image.BILINEAR)
    img_resized.save(output_path, quality=90)

In [5]:
# Données d'entraînement :images et annotations
#Récupérer les annotations pour la détection de zones illustrées
# Raw dataset = dataset avec des annotations pour zones illustrées, avec pages blanches à filtrer
labels_dir = os.path.join(EXPERIMENTS, 'yolo_doc_ill_detect', 'raw_dataset', 'labels')
od_training_dataset = os.path.join(EXPERIMENTS, 'yolo_doc_ill_detect', 'raw_dataset', 'img')


In [6]:
# YOLO experiment dir
yolo_dir = os.path.join(EXPERIMENTS, 'yolo_doc_cls')
colab_yolo_dir = '/content/yolo/data'
#data_dir = os.path.join(yolo_dir, 'data') en local
data_dir = colab_yolo_dir

train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
os.makedirs(train_dir, exist_ok = True)
os.makedirs(val_dir, exist_ok = True)

In [ ]:
# Check chemins à supprimer
print('yolo dir :', yolo_dir)
print('labels dir :', labels_dir)
print('train dir :', train_dir)
print('val dir :', val_dir)


yolo dir : /content/drive/MyDrive/dataHN/experiments/yolo_doc_cls
labels dir : /content/drive/MyDrive/dataHN/experiments/yolo_doc_ill_detect/raw_dataset/labels
train dir : /content/yolo/data/train
val dir : /content/yolo/data/val


In [ ]:
'''
Créer un dataset d'entraînement pour la classification des pages illustrées & non illustrées
en utilisant annotations pour détection d'objet
'''

ill=[]
no_ill=[]

for label_file in os.listdir(labels_dir):
    label_path = os.path.join(labels_dir, label_file)
    with open(label_path, encoding='utf-8') as f:
        txt = f.read()
    if txt.strip() == '':
        no_ill.append(label_file)
    else:
        ill.append(label_file)




In [ ]:
# Vérifier équilibre des classes
print(f"Images with illustration (ill): {len(ill)}")
print(f"Images without illustration (no_ill): {len(no_ill)}")

Images with illustration (ill): 423
Images without illustration (no_ill): 514


In [ ]:



for group, label in [(no_ill, 'no_ill'), (ill, 'ill')]:
    images = [file.replace('.txt', '.jpg') for file in group]
    train_img, val_img = train_test_split(images, test_size=0.2, random_state=42)

    train_subfolder = os.path.join(train_dir, label)
    val_subfolder = os.path.join(val_dir, label)
    os.makedirs(train_subfolder, exist_ok=True)
    os.makedirs(val_subfolder, exist_ok=True)

    # Train
    for img in tqdm(train_img, desc=f'{label} -> train'):
        source = os.path.join(od_training_dataset, img)
        if os.path.exists(source):
            dst_path = os.path.join(train_subfolder, img)
            resize_save_img(source, dst_path, target_size=640)

    # Val
    for img in tqdm(val_img, desc=f'{label} -> val'):
        source = os.path.join(od_training_dataset, img)
        if os.path.exists(source):
            dst_path = os.path.join(val_subfolder, img)
            resize_save_img(source, dst_path, target_size=640)

ill -> val:   5%|▍         | 4/85 [00:03<01:12,  1.12it/s]/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (131542390 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
ill -> val: 100%|██████████| 85/85 [01:16<00:00,  1.11it/s]


# Entraînement YOLO

## Config

In [7]:
model_name = 'yolo11s-cls.pt'
num_training = 9
training_dir = os.path.join(yolo_dir, 'training')

project = os.path.join(training_dir, 'runs')
name = f'run{num_training}_{model_name.split(".")[0]}'


In [8]:
name = f'run{num_training}_{model_name.split(".")[0]}_img_resized'


In [ ]:
#config = os.path.join(training_dir, 'config.yaml')
epochs = 30
batch = 16
dropout = 0.1
imgsz=640

In [ ]:
model = YOLO(model_name)
results = model.train(data = data_dir,
                      epochs=epochs,
                      batch = batch,
                      dropout=dropout,
                      project = project,
                      name = name
                      )

Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo/data, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=run9_yolo11s-cls_img_resized, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose

train: Scanning /content/yolo/data/train... 749 images, 0 corrupt: 100%|██████████| 749/749 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 482.7±202.9 MB/s, size: 47.1 KB)


val: Scanning /content/yolo/data/val... 188 images, 0 corrupt: 100%|██████████| 188/188 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/dataHN/experiments/yolo_doc_cls/training/runs/run9_yolo11s-cls_img_resized
Starting training for 30 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

      30/30     0.879G    0.02521         13        224: 100%|██████████| 47/47 [00:08<00:00,  5.72it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 6/6 [00:00<00:00,  8.44it/s]

                   all      0.973          1



30 epochs completed in 0.077 hours.
Optimizer stripped from /content/drive/MyDrive/dataHN/experiments/yolo_doc_cls/training/runs/run9_yolo11s-cls_img_resized/weights/last.pt, 11.0MB
Optimizer stripped from /content/drive/MyDrive/dataHN/experiments/yolo_doc_cls/training/runs/run9_yolo11s-cls_img_resized/weights/best.pt, 11.0MB

Validating /content/drive/MyDrive/dataHN/experiments/yolo_doc_cls/training/runs/run9_yolo11s-cls_img_resized/weights/best.pt...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s-cls summary (fused): 47 layers, 5,436,690 parameters, 0 gradients, 12.0 GFLOPs
train: /content/yolo/data/train... found 749 images in 2 classes ✅ 
val: /content/yolo/data/val... found 188 images in 2 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 6/6 [00:00<00:00,  9.11it/s]


                   all      0.995          1
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/dataHN/experiments/yolo_doc_cls/training/runs/run9_yolo11s-cls_img_resized


# Inference

In [ ]:
import shutil

#Transférer données sur espace de stockage colab 
source_folder = '/content/drive/MyDrive/dataHN/data/raw_selected/doc_imgs_640'
colab_data_dir = '/content/yolo/inference/doc_imgs_resized'

shutil.copytree(source_folder, colab_data_dir, dirs_exist_ok=True)

'/content/yolo/inference/doc_imgs_resized'

In [10]:
model_best_path = os.path.join(project, name, 'weights','best.pt')
model_best_path = '/content/drive/MyDrive/dataHN/experiments/yolo_doc_cls/training/runs/run9_yolo11s-cls_img_resized/weights/best.pt'


In [11]:
#predict_data = DOC_IMGS # en local
predict_data = colab_data_dir
predict_images = [os.path.join(predict_data, img) for img in os.listdir(predict_data)]
len(predict_images)

4818

In [12]:
predicts_dir = os.path.join(yolo_dir, 'predicts')
project = os.path.join(predicts_dir, 'runs')
name = f'run{num_training}_{model_name.split(".")[0]}'
csv_dir = os.path.join(predicts_dir, 'csv')
os.makedirs(csv_dir, exist_ok=True)
results_csv = os.path.join(csv_dir, f'{name}_prediction_results.csv')

In [13]:
import pandas as pd
import numpy as np
import torch
import os

# Split image list into 4 sections
sections = np.array_split(predict_images, 4)
sections = [list(section) for section in sections]

model_best = YOLO(model_best_path)


for i, section_data in enumerate(sections):
    print(f"Processing section {i+1}/4...")

    results = model_best.predict(section_data, project=project, name = name, batch=4)

    all_results = []
    for result in results:
        probs = result.probs
        top1_idx = probs.top1
        top1_conf = probs.top1conf.item()
        class_names = result.names

        all_results.append({
            'image': result.path,
            'predicted_class': class_names[top1_idx],
            'confidence': top1_conf
        })

    df_section = pd.DataFrame(all_results)
    df_section.to_csv(results_csv, mode='a', index=False, header=(i == 0))

    torch.cuda.empty_cache()  # still useful if you're using GPU


Processing section 1/4...

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 224x224 no_ill 1.00, ill 0.00, 0.6ms
1: 224x224 ill 0.97, no_ill 0.03, 0.6ms
2: 224x224 no_ill 1.00, ill 0.00, 0.6ms
3: 224x224 ill 1.00, no_ill 0.00, 0.6ms
4: 224x224 ill 1.00, no_ill 0.00, 0.6ms
5: 224x224 ill 1.00, no_ill 0.00, 0.6ms
6: 224x224 ill 0.56, no_ill 0.44, 0.6ms
7: 224x224 ill 1.00, no_ill 0.00, 0.6ms
8: 224x224 no_ill 1.00, ill 0.00, 0.6ms
9: 224x224 ill 1.00, no_ill 0.00, 0.6ms
10: 224x224 ill 1.00, no_ill 0

In [14]:
all_results[1]

{'image': np.str_('/content/yolo/inference/doc_imgs_resized/Goury-Jones_1845_167.jpg'),
 'predicted_class': 'ill',
 'confidence': 1.0}

## Sort results for visualization

In [ ]:
import os
import pandas as pd
from PIL import Image

csv_path = results_csv  # chemin vers le fichier CSV
output_dir = os.path.join(predicts_dir, "sorted_results")
resize_target = 256  # taille cible pour verification

ill_dir = os.path.join(output_dir, "ill")
no_ill_dir = os.path.join(output_dir, "no_ill")
os.makedirs(ill_dir, exist_ok=True)
os.makedirs(no_ill_dir, exist_ok=True)

df = pd.read_csv(csv_path)

for _, row in df.iterrows():
    img_path = row["image"]
    predicted_class = row["predicted_class"]

    if not os.path.isfile(img_path):
        print(f"Image introuvable : {img_path}")
        continue

    try:
        # Détermination du dossier de destination
        filename = os.path.basename(img_path)
        if predicted_class == "ill":
            save_path = os.path.join(ill_dir, filename)
        else:
            save_path = os.path.join(no_ill_dir, filename)

        # Redimensionner et sauvegarder avec la fonction existante
        resize_save_img(img_path, save_path, target_size=resize_target)

    except Exception as e:
        print(f"Erreur avec {img_path} : {e}")